# dpp vs uniform on identical sorted vs unsorted pods with identical histories

This notebook will generate pods sampled from uniform psi and will create both sorted and unsorted
sumstats. We do this by executing two simultaneous instances of msbayes (it samples time as a seed for
the random number generator so two instances started at the same time will produce the same histories).


In [42]:
import os
import sys
import time
import subprocess

#msbayes paths
MSBAYES_ROOTDIR="/Volumes/WorkDrive/msbayes-buffering/hickerlab-repository/msbayes-buffering/"
MSBAYES_EXECDIR=MSBAYES_ROOTDIR+"src/"

# Binaries
MSBAYES_BIN=MSBAYES_EXECDIR+"msbayes.pl"

# Top level analysis directories
MSBAYES_DATADIR=MSBAYES_ROOTDIR+"data/"

# Dirichlet priors/confs
DIRICHLET_OUT=MSBAYES_DATADIR+"dirichlet/"
DIRICHLET_PRIORS_DIR=DIRICHLET_OUT+"priors/"
DIRICHLET_CONF_DIR=DIRICHLET_OUT+"conf/"

# Uniform priors/confs
UNIFORM_OUT=MSBAYES_DATADIR+"pods/"
UNIFORM_PRIORS_DIR=UNIFORM_OUT+"priors/"
UNIFORM_CONF_DIR=UNIFORM_OUT+"conf/"

# 2x2 pods draws and results
TWOxTWO_OUT=MSBAYES_DATADIR+"2x2/"
TWOxTWO_PODS_DIR=TWOxTWO_OUT+"pods_draws/"
TWOxTWO_RESULTS_DIR=TWOxTWO_OUT+"results/"

os.chdir(MSBAYES_EXECDIR)

In [36]:
## Define the msbayes priors command
def do_priors(num, outname, conf_file):
    for i in range(num):
        ## Switch on whether this is the first iteration
        out = ""
        if i == 0:
            out = " -o "
        else:
            out = " | tail -n 1 >> "
        
        seed = str(time.time())
        
        os.chdir(MSBAYES_EXECDIR)
        args_sort = "" \
            + " -s 7 " \
            + " -r 1 " \
            + " -c " + conf_file \
            + " -S " + seed \
            + out + outname+"_sort"
        args_usort = "" \
            + " -s 0 " \
            + " -r 1 " \
            + " -c " + conf_file \
            + " -S " + seed \
            + out + outname+"_usort"

        with open('tmp.sh', 'w') as f:
            f.write("#!/bin/bash\n")
            f.write(MSBAYES_BIN + args_sort + " & \n")
            f.write(MSBAYES_BIN + args_usort + "\n")

        try:
            time.sleep(2)
            os.system("chmod 777 tmp.sh; ./tmp.sh")
    #        pid = subprocess.Popen(["pwd", " > /tmp/rpt.txt"])
    #        pid1 = subprocess.Popen([MSBAYES_BIN, args_sort]).pid
    #        pid2 = subprocess.Popen([MSBAYES_BIN, args_usort]).pid
        except Exception as inst:
            print(inst)

In [51]:
do_priors(100, TWOxTWO_PODS_DIR+"pods", UNIFORM_CONF_DIR+"conf_nobuffer_psi18_ne0.5_taumax0.75.txt")

# Do the PODS

In [52]:
## Go to the msbayes working directory
os.chdir(MSBAYES_EXECDIR)

## Output
DPP_SORTED_OUT = TWOxTWO_RESULTS_DIR + "dpp_sort/"
DPP_UNSORTED_OUT = TWOxTWO_RESULTS_DIR + "dpp_usort/"
UNIF_SORTED_OUT = TWOxTWO_RESULTS_DIR + "unif_sort/"
UNIF_UNSORTED_OUT = TWOxTWO_RESULTS_DIR + "unif_usort/"

## PODS files
SORTED_PODS = TWOxTWO_PODS_DIR + "pods_sort"
UNSORTED_PODS = TWOxTWO_PODS_DIR + "pods_usort"

## DPP/Uniform Configs
DPP_CONF = DIRICHLET_CONF_DIR + "conf_dirichlet_buffer0.txt"
UNIF_CONF = UNIFORM_CONF_DIR + "conf_nobuffer_psi18_ne0.5_taumax0.75.txt"

## All priors files
DPP_SORTED_PRIOR = DIRICHLET_PRIORS_DIR + "buffer0-sorted/dpp-sorted.prior"
DPP_UNSORTED_PRIOR = DIRICHLET_PRIORS_DIR + "buffer0/buffer0.prior"
UNIF_SORTED_PRIOR = UNIFORM_PRIORS_DIR + "new_priors/sorted_buffer0.prior"
UNIF_UNSORTED_PRIOR = UNIFORM_PRIORS_DIR + "new_priors/unsorted_buffer0.prior"

## Dict for mapping pods to priors
runs = ["dpp_sort", "dpp_usort", "unif_sort", "unif_usort"]
runs_dict = {}
for i in runs:
    runs_dict[i] = {}

## Populate the runs_dict
runs_dict["dpp_sort"]["priors"] = DPP_SORTED_PRIOR
runs_dict["dpp_sort"]["pods"] = SORTED_PODS
runs_dict["dpp_sort"]["conf"] = DPP_CONF
runs_dict["dpp_sort"]["outdir"] = DPP_SORTED_OUT

runs_dict["dpp_usort"]["priors"] = DPP_UNSORTED_PRIOR
runs_dict["dpp_usort"]["pods"] = UNSORTED_PODS
runs_dict["dpp_usort"]["conf"] = DPP_CONF
runs_dict["dpp_usort"]["outdir"] = DPP_UNSORTED_OUT

runs_dict["unif_sort"]["priors"] = UNIF_SORTED_PRIOR
runs_dict["unif_sort"]["pods"] = SORTED_PODS
runs_dict["unif_sort"]["conf"] = UNIF_CONF
runs_dict["unif_sort"]["outdir"] = UNIF_SORTED_OUT

runs_dict["unif_usort"]["priors"] = UNIF_UNSORTED_PRIOR
runs_dict["unif_usort"]["pods"] = UNSORTED_PODS
runs_dict["unif_usort"]["conf"] = UNIF_CONF
runs_dict["unif_usort"]["outdir"] = UNIF_UNSORTED_OUT

NPODS = 100
for run_name, params in runs_dict.items():
    print("Doing - " + run_name)
    print("Params - {}".format(params))

    ## Clean up any existing previous results files or else the results are wacky
    results_files = ["results_psi_omega_et_tol.out", "results_psi.out", \
             "results_omega_unadjusted.out", "results_omega.out", "all_pods.obs"]
    for f in results_files:
        if os.path.exists(f):
            os.path.remove(f)
    
    cmd = "./bu_doPODS.sh" \
        + " -p " + params["priors"] \
        + " -c " + params["conf"] \
        + " -n " + str(NPODS) \
        + " -e " + params["pods"]

    print(cmd)
    os.system(cmd)

    ## When you're done, save the output files
    for res in results_files:
        os.rename(res, params["outdir"]+res)

Doing - dpp_usort
Params - {'priors': '/Volumes/WorkDrive/msbayes-buffering/hickerlab-repository/msbayes-buffering/data/dirichlet/priors/buffer0/buffer0.prior', 'pods': '/Volumes/WorkDrive/msbayes-buffering/hickerlab-repository/msbayes-buffering/data/2x2/pods_draws/pods_usort', 'conf': '/Volumes/WorkDrive/msbayes-buffering/hickerlab-repository/msbayes-buffering/data/dirichlet/conf/conf_dirichlet_buffer0.txt', 'outdir': '/Volumes/WorkDrive/msbayes-buffering/hickerlab-repository/msbayes-buffering/data/2x2/results/dpp_usort/'}
./bu_doPODS.sh -p /Volumes/WorkDrive/msbayes-buffering/hickerlab-repository/msbayes-buffering/data/dirichlet/priors/buffer0/buffer0.prior -c /Volumes/WorkDrive/msbayes-buffering/hickerlab-repository/msbayes-buffering/data/dirichlet/conf/conf_dirichlet_buffer0.txt -n 100 -e /Volumes/WorkDrive/msbayes-buffering/hickerlab-repository/msbayes-buffering/data/2x2/pods_draws/pods_usort
Doing - unif_sort
Params - {'priors': '/Volumes/WorkDrive/msbayes-buffering/hickerlab-rep